# Importing Libraries

---



In [ ]:
from __future__ import print_function

import tensorflow.keras

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, GlobalAveragePooling2D

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Model

**Defining Parameters**

In [ ]:
Batch_Size = 128
Num_of_Iterations = 50
Num_of_Classes = 5
Num_of_Channels = 3
image_dimn = [32,32]
input_image_shape = [32,32,3]

# Loading Data

---



In [ ]:
!7z x Trainingless.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 141541935 bytes (135 MiB)

Extracting archive: Trainingless.zip
 81% 4096 Open              --
Path = Trainingless.zip
Type = zip
Physical Size = 141541935

  0%     16% 817 - Trainingless/C1/831.jpg                                   30% 1511          42% 2123 - Trainingless/Cdoop/586.jpg                                       53% 2641 - Trainingless/C1/576.jpg                                    63% 3123 - Trainingless/C1/380.jpg

In [ ]:
!7z x Testless.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 39336323 bytes (38 MiB)

Extracting archive: Testless.zip
--
Path = Testless.zip
Type = zip
Physical Size = 39336323

  0%     59% 744 - Testless/L/106.jpg                             Everything is Ok

Files: 1250
Size:       39116534
Compressed: 39336323


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
)

train_generator = train_datagen.flow_from_directory(
    directory="/content/Trainingless/",
    target_size=image_dimn,
    color_mode="rgb",
    batch_size=Batch_Size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 5000 images belonging to 6 classes.


In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1/255,
)

test_generator = test_datagen.flow_from_directory(
    directory="/content/Testless/",
    target_size=image_dimn,
    color_mode="rgb",
    batch_size=Batch_Size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1250 images belonging to 6 classes.


# Building Model Using Transfer Learning (PreTrained VGG16)

---



**Building and Training the Model**

In [ ]:
from tensorflow.keras.applications import VGG16

#Using VGG16 as our Backbone
base_model2 = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=input_image_shape
)

x = base_model2.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

#Freezing Layers of VGG16
model2 = Model(inputs=base_model2.input, outputs=predictions)
for layer in base_model2.layers:
    layer.trainable = False

#Compiling our Model
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#Model Check point to save our best Model
checkpoint_filepath = '/content/drive/MyDrive/weights/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#Training our Model
hist = model2.fit(train_generator, validation_data=(test_generator), batch_size = Batch_Size, epochs = 50, shuffle=True, callbacks=[model_checkpoint_callback])

#Loading weights from the best model
model2.load_weights(checkpoint_filepath)

Epoch 1/50
40/40 [==============================] - 15s 315ms/step - loss: 0.8820 - accuracy: 0.6526 - val_loss: 0.3933 - val_accuracy: 0.9008
Epoch 2/50
40/40 [==============================] - 11s 282ms/step - loss: 0.1827 - accuracy: 0.9354 - val_loss: 0.2361 - val_accuracy: 0.9256
Epoch 3/50
40/40 [==============================] - 11s 264ms/step - loss: 0.1139 - accuracy: 0.9586 - val_loss: 0.2335 - val_accuracy: 0.9184
Epoch 4/50
40/40 [==============================] - 11s 268ms/step - loss: 0.0764 - accuracy: 0.9704 - val_loss: 0.1740 - val_accuracy: 0.9472
Epoch 5/50
40/40 [==============================] - 11s 263ms/step - loss: 0.0883 - accuracy: 0.9664 - val_loss: 0.3930 - val_accuracy: 0.8816
Epoch 6/50
40/40 [==============================] - 11s 277ms/step - loss: 0.0671 - accuracy: 0.9758 - val_loss: 0.1988 - val_accuracy: 0.9416
Epoch 7/50
40/40 [==============================] - 11s 263ms/step - loss: 0.0653 - accuracy: 0.9738 - val_loss: 0.2014 - val_accuracy: 0.9456

Testing the Model

In [ ]:
scores = model2.evaluate(test_generator, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', 100*scores[1])

10/10 [==============================] - 2s 214ms/step - loss: 0.2309 - accuracy: 0.9544
Test loss: 0.2309301495552063
Test accuracy: 95.44000029563904


# Model Accuracy : 95.44 %